In [1]:
!pip install tensorflow tensorflow_datasets

     |████████████████████████████████| 5.4 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 10.3 MB/s eta 0:00:01     |████████████████▌               | 1.6 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 146 kB 15.9 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 224 kB 10.2 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21485 sha256=7f3a7cf4bb1f94a6bf86d079f1becdbe029097b8150b9e59c341da6f90dbb33e
  Stored in directory: /root/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise


In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the NYU depth V2 dataset
dataset, info = tfds.load('nyu_depth_v2', split='train', with_info=True)

# Preprocess the dataset
def preprocess_data(data):
    image = tf.image.resize(data['image'], (224, 224))  # Resize image to match model input shape
    image = tf.cast(image, tf.float32) / 255.0
    depth = tf.cast(data['depth'], tf.float32) / 10.0  # Normalize depth values
    return image, depth

# Apply preprocessing to the dataset using from_generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: dataset,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32)
    )
).map(preprocess_data)

# Split the dataset into training and validation sets
train_size = int(0.8 * info.splits['train'].num_examples)
train_dataset = train_dataset.take(train_size)
val_dataset = train_dataset.skip(train_size)

# Configure batch size and prefetching
batch_size = 2
train_dataset = train_dataset.batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

2023-06-22 16:36:46.031669: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_629/4258777829.py", line 5, in <module>
    dataset, info = tfds.load('nyu_depth_v2', split='train', with_info=True)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_datasets/core/logging/__init__.py", line 169, in __call__
    return function(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_datasets/core/load.py", line 640, in load
    _download_and_prepare_builder(dbuilder, download, download_and_prepare_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_datasets/core/load.py", line 499, in _download_and_prepare_builder
    dbuilder.download_and_prepare(**download_and_prepare_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_datasets/core/logging/__init__.py", line 169, in __call__
    return function(*arg

In [ ]:
# Load the MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224, 224, 3))  # Update input shape

# Freeze the base model's layers
base_model.trainable = False

# Add a custom output layer for depth prediction
output_layer = tf.keras.layers.Conv2D(1, 3, activation='relu', padding='same')(base_model.output)

# Create the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=output_layer)

# Define loss function and metrics
loss_fn = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsoluteError()]

# Compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=metrics)

In [ ]:
# Train the model
epochs = 1
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

In [ ]:
# Save the trained model
model.save('mobilenetv2_nyu_depth_model.h5')